**Imports**

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import pandas  # used to load metadata table
import numpy as np
import sys




import numpy as np
import os
from Xana import Xana
from Xana.misc.makemask import masker
from matplotlib.colors import LogNorm


import pandas as pd
import pickle
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.optimize import curve_fit
from matplotlib import rc

import matplotlib.pyplot as plt
import extra_data as ex

import midtools
from midtools.scheduler import Scheduler
from midtools.scheduler import start_midtools

## Setting Paths

Specify the output directory for the results (`BASE_DIR`) and for the SLURM logs (`JOB_DIR`). The later will be used for monitoring the progress of the jobs.

In [2]:
JOB_DIR = "./analyzed_runs_photonised_ADU8p9"
BASE_DIR = "./analyzed_runs_photonised_ADU8p9/"

Midtools is shipped with a mask but you probably want to make your own at a later stage. The mask needs to by stored as an `npy` file and must have the shape (16,512,128).

In [3]:
ls ../shared/masks/

darkmask_r0032_00.npy  mask_1611.npy      mask_r0057_00.npy  mask_r0057_02.npy
mask_1611_2.npy        mask_r0041_00.npy  mask_r0057_01.npy  mask_r0057_03.npy


In [4]:
maskfile ='../shared/masks/mask_r0057_02.npy'
#maskfile ='../shared/masks/darkmask_r0032_00.npy'
maskfile ='../shared/masks/mask_1611_2.npy'

print(f"Mask exists: {os.path.exists(maskfile)}.")
print(f"Maskfile is: {maskfile}")
print(f"Maskshape is: {np.load(maskfile).shape}")

Mask exists: True.
Maskfile is: ../shared/masks/mask_1611_2.npy
Maskshape is: (16, 512, 128)


## Setupfiles

Setupfiles are used to set most of the parameters. Next to each result-file the setupfile is stored that was used to generate the results.

**Change the maskfile!**: to the maskfile from the last cell.

In [5]:
%%writefile base_setupfile.yml
mask: ../shared/masks/mask_r0057_02.npy
photon_energy: 9
pixel_size: 200
beamcenter: 595.0, 680.0
#adu_per_photon:
quadrant_positions:
  dx: 0.0
  dy: 0.0
  q1:
  - -542.0
  - 659.9
  q2:
  - -608.0
  - -35.0
  q3:
  - 534.0
  - -221.0
  q4:
  - 588.0
  - 474.0
sample_detector: 7
saxs_opt:
    integrate: 1d
    nqbins: 256
#     nphibins: 128
xpcs_opt:
  norm: symmetric
  q_range:
    nsteps: 16
    q_first: 0.2
    q_last: 0.7
    scale: linear
#   phi_range:
#     phi_width: 45
#     phi_first: 0
#     phi_last: 180
#     scale: linear
calib_opt:
    dropletize: True       
slurm_opt:
    njobs: 8
    nprocs: 4
    memory: 400GB
    partition: upex

Overwriting base_setupfile.yml


In [6]:
!cat base_setupfile.yml

mask: ../shared/masks/mask_r0057_02.npy
photon_energy: 9
pixel_size: 200
beamcenter: 595.0, 680.0
#adu_per_photon:
quadrant_positions:
  dx: 0.0
  dy: 0.0
  q1:
  - -542.0
  - 659.9
  q2:
  - -608.0
  - -35.0
  q3:
  - 534.0
  - -221.0
  q4:
  - 588.0
  - 474.0
sample_detector: 7
saxs_opt:
    integrate: 1d
    nqbins: 256
#     nphibins: 128
xpcs_opt:
  norm: symmetric
  q_range:
    nsteps: 16
    q_first: 0.2
    q_last: 0.7
    scale: linear
#   phi_range:
#     phi_width: 45
#     phi_first: 0
#     phi_last: 180
#     scale: linear
calib_opt:
    dropletize: True       
slurm_opt:
    njobs: 8
    nprocs: 4
    memory: 400GB
    partition: upex


The metadata of our MID beamtime in fall 2020 is available, but usually you need to figure out all the parameters yourself or ask your local contact. As an example, we analyze run 280 of experiment 2718, which is nanoparticles in water. 

In [7]:
proposal = 3094
run_number = 216
# Both uncalibrated (raw) and calibrated (proc) data is available
# for the AGIPD. With `data="all" we select raw by default, but proc
# data for all sources that have been calibrated by the calibration pipeline.
_run = ex.open_run(proposal=proposal, run=run_number,data='all')
#_run.info()


all_train_ids=_run.train_ids

`starts_midtools` submits a job to run the analysis. The keyword argument `test` can be set to see the batch file without actually submitting the job. The parameters in the next cell have been set for you. Check the output and then change `TEST=False` to submit the job.

**Note:** The data direcotry (datdir) in the metadata table is not correct, but the datdir in the next cell is.

In [27]:
proposal = 3094

# Both uncalibrated (raw) and calibrated (proc) data is available
# for the AGIPD. With `data="all" we select raw by default, but proc
# data for all sources that have been calibrated by the calibration pipeline.
runs_saxs=[97,79,81,85,87,77,63,259,255,149,146,143,140,131,189,201]



# to make sure that the analysis goes through the runs are divided in slices of trains_per_file(=500) trains 


for run_number in range(152,177):
        _run = ex.open_run(proposal=proposal, run=run_number,data='all')
        #_run.info()
        all_train_ids=_run.train_ids
        n_totTrain=len(all_train_ids)
        trains_per_file=500


        slices=int(np.ceil(n_totTrain/trains_per_file))

        print(n_totTrain)
        print(slices)

        TEST = False
        for i in range(slices):
                start_midtools(
                        proposal=proposal,
                        run_number=run_number,
                        setupfile='base_setupfile.yml',
                        pulses_per_train=100,
                        saxs=False,
                        xpcs=True,
                        datdir='/gpfs/exfel/exp/MID/202202/p003094/proc/',
                        first_cell=2,
                        pulse_step=1,
                        last_train=i*trains_per_file+trains_per_file,
                        first_train=i*trains_per_file,
                        test=TEST,
                        jobdir=JOB_DIR,
                        base_dir=BASE_DIR,

                )



1264
3
Generating sbatch jobs for run: 152
With arguments: `--out-dir ./analyzed_runs_photonised_ADU8p9/p003094 --pulses-per-train 100 --pulse-step 1 --datdir /gpfs/exfel/exp/MID/202202/p003094/proc/ --last-train 500 --first-cell 2`
Generating and submitting sbatch for job 2023-01-16T18-45-57-665-run152
Submitted batch job 199159
Generating sbatch jobs for run: 152
With arguments: `--out-dir ./analyzed_runs_photonised_ADU8p9/p003094 --pulses-per-train 100 --pulse-step 1 --datdir /gpfs/exfel/exp/MID/202202/p003094/proc/ --first-train 500 --last-train 1000 --first-cell 2`
Generating and submitting sbatch for job 2023-01-16T18-45-57-979-run152
Submitted batch job 199160
Generating sbatch jobs for run: 152
With arguments: `--out-dir ./analyzed_runs_photonised_ADU8p9/p003094 --pulses-per-train 100 --pulse-step 1 --datdir /gpfs/exfel/exp/MID/202202/p003094/proc/ --first-train 1000 --last-train 1500 --first-cell 2`
Generating and submitting sbatch for job 2023-01-16T18-45-57-074-run152
Submit

In [52]:
TEST = False

start_midtools(
          proposal,
          run_number,
          setupfile='base_setupfile.yml',
          pulses_per_train=100,
          saxs=True,
          xpcs=True,
          datdir='/gpfs/exfel/exp/MID/202202/p003094/proc/',
          first_cell=2,
          pulse_step=1,
          last_train=10,
          first_train=0,
          test=TEST,
          jobdir=JOB_DIR,
          base_dir=BASE_DIR,

)

Generating sbatch jobs for run: 214
With arguments: `--out-dir ./analyzed_runs_photonised_ADU8p9/p003094 --pulses-per-train 100 --pulse-step 1 --datdir /gpfs/exfel/exp/MID/202202/p003094/proc/ --last-train 10 --first-cell 2`
Generating and submitting sbatch for job 2022-12-09T03-54-31-114-run214
Submitted batch job 13596781


## Monitor Jobs

Use the following set of widgets to monitor your analysis jobs. Start by clicking on `Update Job Table`. `list jobs` will list all your SLURM jobs.

In [10]:
!ls ./analyzed_runs_photonised_ADU8p9/p003094/

c_120_abs1_RT_errorbars.npz   r0098-setup_016.yml    r0177-setup_001.yml
c_120_abs1_RT.npz	      r0098-setup_017.yml    r0177-setup_002.yml
c_120_abs2_RT_errorbars.npz   r0098-setup_018.yml    r0178-analysis_000.h5
c_120_abs2_RT.npz	      r0098-setup_019.yml    r0178-analysis_002.h5
c_120_abs4_RT_errorbars.npz   r0104-analysis_000.h5  r0178-analysis_003.h5
c_120_abs4_RT.npz	      r0104-analysis_001.h5  r0178-setup_000.yml
c_120_absp6_RT_errorbars.npz  r0104-setup_000.yml    r0178-setup_002.yml
c_120_absp6_RT.npz	      r0104-setup_001.yml    r0178-setup_003.yml
c_15_abs10_RT_errorbars.npz   r0105-analysis_000.h5  r0179-analysis_001.h5
c_15_abs10_RT.npz	      r0105-analysis_001.h5  r0179-analysis_002.h5
c_15_abs2_RT_errorbars.npz    r0105-analysis_003.h5  r0179-analysis_003.h5
c_15_abs2_RT.npz	      r0105-analysis_004.h5  r0179-setup_001.yml
c_60_abs10_23C_errorbars.npz  r0105-analysis_007.h5  r0179-setup_002.yml
c_60_abs10_23C.npz	      r0105-analysis_008.h5  r0179-setup_003.yml
c_60_ab

In [33]:
scheduler = Scheduler("./analyzed_runs_photonised_ADU8p9")  


In [34]:
scheduler.gui()

Output()